In [1]:
from nytimesarticle import articleAPI
api = articleAPI('2462ecddebb043db8fce29ca0d9b9b66')
import pandas as pd
import time
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
#getting buzzfeed data
buzz_df = pd.read_excel('./practice.xlsx')
buzz_df = buzz_df[0:100]




In [3]:
#functions for getting and parsing NYT articles

#Function to parse the results of the API query into a list of dictionaries where each dictionary represents a different article


def parser(articles):
    news = []
    for article in articles['response']['docs']:
        dic = {}
        dic['date'] = article['pub_date'][0:10]
        dic['Headline'] = article['headline']['main']
        news.append(dic)
    return news 

# This function performs a search of the NYTimes api for a page range and then returns the articles as a list of dictionaries 


def get_articles():
    headlines = []
    for i in range(0,10):
        article = api.search(begin_date = 20140401, end_date = 20140428,sort = 'newest',page = str(i), fl = ['headline','pub_date'])
        try:
            article = parser(article)
            headlines += article
            time.sleep(1)
        except KeyError:
            print article
    return headlines

In [4]:
#getting nyt data

nyt = get_articles()
nyt_df = pd.DataFrame(nyt)
nyt_df['Source'] = 'New York Times'
nyt_df = nyt_df.drop('date', 1)
nyt_df



#merging data together into one table Question: Does concatenating mess up the indices somehow??

headlines = pd.concat([buzz_df,nyt_df]).drop_duplicates().reset_index(drop=True)
headlines = headlines.reindex(np.random.permutation(headlines.index))
headlines








,Headline,Source
177,Pictures of the Day: Egypt and Elsewhere,New York Times
192,'To Kill a Mockingbird' Author Agrees to e-Boo...,New York Times
105,Chinese Premier Calls for Investment in Key Re...,New York Times
18,Riot Square Sanctificare,BuzzFeed
53,The “How I Met Your Mother” Ending Fans Deserved,BuzzFeed
112,China Telecom Profit Up 17.9 Percent,New York Times
91,What Sea Creature Are You?,BuzzFeed
119,Penpix of Likely Ecuador Squad for World Cup F...,New York Times
148,EnergySolutions Sues NDA After Losing $12 Bill...,New York Times
108,Gray Throws 3-Hitter for A's in 4-0 Win at Texas,New York Times


In [10]:

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1,stop_words='english')
vectorizer.fit(headlines['Headline'].values)
X = vectorizer.transform(headlines['Headline'].values)
y = headlines['Source'].values






In [9]:


from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

examples = ["7 Signs You're Still Not Over The Breakup","Trump Details Plans to Deport Millions of Immigrants"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions




NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [11]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('classifier', MultinomialNB())
    ])

pipeline.fit(headlines['Headline'].values, headlines['Source'].values)
pred = pipeline.predict(examples)
pred




array([u'BuzzFeed', u'New York Times'], 
      dtype='<U14')

In [ ]:
from sklearn.model_selection import GridSearchCV
params_dict = {
    'vectorizer__min_df': (1,5,10,15),
    'classifier__alpha': np.linspace(0.1,1,10)
}

gridsearch = GridSearchCV(pipeline,params_dict)
gridsearch.fit(headlines['Headline'].values,headlines['Source'].values)

